In [9]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix

In [10]:
pd.options.display.max_columns=100
pd.options.display.max_rows=100

In [11]:
!ls ../../../data/

donor1_atac_processed.h5ad  s1d2_atac_processed.h5ad  s2d4_atac_processed.h5ad
donor1_rna_processed.h5ad   s1d2_rna_processed.h5ad   s2d4_rna_processed.h5ad
merged			    s1d3_atac_processed.h5ad  s2d5_atac_processed.h5ad
multiome		    s1d3_rna_processed.h5ad   s2d5_rna_processed.h5ad


# Input data

Load data via:

ATAC:
```
aws s3 cp s3://openproblems-bio/neurips2021/site1/production/donor01.lot3054455/atac/processed/donor1_atac_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site1/production/donor02.lot3054129/atac/processed/s1d2_atac_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site1/production/donor03.lot3054716/atac/processed/s1d3_atac_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site2/production/donor04.lot3051683/atac/processed/s2d4_atac_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site2/production/donor05.lot3054457/atac/processed/s2d5_atac_processed.h5ad .
```

RNA:
```
aws s3 cp s3://openproblems-bio/neurips2021/site1/production/donor01.lot3054455/atac/processed/donor1_rna_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site1/production/donor02.lot3054129/atac/processed/s1d2_rna_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site1/production/donor03.lot3054716/atac/processed/s1d3_rna_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site2/production/donor04.lot3051683/atac/processed/s2d4_rna_processed.h5ad .
aws s3 cp s3://openproblems-bio/neurips2021/site2/production/donor05.lot3054457/atac/processed/s2d5_rna_processed.h5ad .
```


In [12]:
#data file dictionaries
data_files_rna = {
    's1d1':'../../../data/donor1_rna_processed.h5ad',
    's1d2':'../../../data/s1d2_rna_processed.h5ad',
    's1d3':'../../../data/s1d3_rna_processed.h5ad',
    's2d4':'../../../data/s2d4_rna_processed.h5ad',
    's2d5':'../../../data/s2d5_rna_processed.h5ad',
}

data_files_atac = {
    's1d1':'../../../data/donor1_atac_processed.h5ad',
    's1d2':'../../../data/s1d2_atac_processed.h5ad',
    's1d3':'../../../data/s1d3_atac_processed.h5ad',
    's2d4':'../../../data/s2d4_atac_processed.h5ad',
    's2d5':'../../../data/s2d5_atac_processed.h5ad',
}

rna_output_filename = '../../../data/merged/Multiome_gex_processed.h5ad'
atac_output_filename = '../../../data/merged/Multiome_atac_processed.h5ad'

In [13]:
samples = list(data_files_rna.keys())

# Read files

In [14]:
rna_dict = dict()

for k in data_files_rna.keys():
    rna_dict[k] = sc.read(data_files_rna[k])


In [15]:
atac_dict = dict()

for k in data_files_atac.keys():
    atac_dict[k] = sc.read(data_files_atac[k])


# Filter objects if needed

## GEX

In [16]:
rna_dict

{'s1d1': AnnData object with n_obs × n_vars = 6224 × 17813
     obs: 'n_genes_by_counts', 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'leiden_final', 'atac_ann', 'cell_type', 'pseudotime_order_GEX'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable'
     uns: 'hvg', 'neighbors', 'pca', 'rg_res0.5_4sub', 'rg_res0.5_msub3', 'umap'
     obsm: 'X_pca', 'X_umap'
     varm: 'PCs'
     layers: 'counts'
     obsp: 'connectivities', 'distances',
 's1d2': AnnData object with n_obs × n_vars = 6740 × 16946
     obs: 'n_genes_by_counts', 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'leiden_final', 'atac_ann', 'cell_type', 'pseudotime_order_GEX'
     var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'highly_variable'
     uns: 'hvg', 'neighbors', 'pca', 'umap'
     obsm: 'X_pca', 'X_umap'
     varm: 'PCs'
     layers: 'counts'
     obsp: 'connectivities', 'distances',
 's1d3': AnnData object with n_obs × n_vars = 4279 × 15832
  

### uns

In [17]:
#Check in uns
common_keys = set(rna_dict[samples[0]].uns.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(rna_dict[k].uns.keys()))
    all_keys.update(set(rna_dict[k].uns.keys()))
    
diff_keys = all_keys.difference(common_keys)
print(f'To keep: {common_keys}')
print(f'To delete: {diff_keys}')

To keep: {'pca', 'umap', 'hvg', 'neighbors'}
To delete: {'leiden', 'rg_res0.3_1sub', 'rg_res0.3_2sub', 'rg_res0.5_prog_3', 'rg_res0.3_5sub', 'rg_res0.5_prog_2', 'rg_res0.3_6sub', 'rg_res0.3_3sub', 'rg_res0.5_4sub', 'rank_genes_res0.3', 'rg_res0.5_msub3'}


In [18]:
# WATCH OUT... only if you really want to delete all keys mentioned above!

for k1 in samples:
    for k2 in list(diff_keys):
        if k2 in rna_dict[k1].uns:
            del rna_dict[k1].uns[k2]
        
common_keys = set(rna_dict[samples[0]].uns.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(rna_dict[k].uns.keys()))
    all_keys.update(set(rna_dict[k].uns.keys()))
    
diff_keys = all_keys.difference(common_keys)
if len(diff_keys) == 0:
    print('Deleted all non-shared keys!')
else:
    print(f'Still have the following keys left over: {diff_keys}')

Deleted all non-shared keys!


### var

In [19]:
#Check in var
common_keys = set(rna_dict[samples[0]].var.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(rna_dict[k].var.keys()))
    all_keys.update(set(rna_dict[k].var.keys()))
    
diff_keys = all_keys.difference(common_keys)
print(f'To keep: {common_keys}')
print(f'To delete: {diff_keys}')

To keep: {'gene_ids', 'feature_types', 'n_cells', 'genome', 'highly_variable'}
To delete: {'log1p_mean_counts', 'total_counts', 'dispersions_norm', 'mean_counts', 'pct_dropout_by_counts', 'log1p_total_counts', 'n_cells_by_counts', 'means', 'mt', 'dispersions'}


In [20]:
# WATCH OUT... only if you really want to delete all keys mentioned above!

for k1 in samples:
    for k2 in list(diff_keys):
        if k2 in rna_dict[k1].var:
            del rna_dict[k1].var[k2]
        
common_keys = set(rna_dict[samples[0]].var.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(rna_dict[k].var.keys()))
    all_keys.update(set(rna_dict[k].var.keys()))
    
diff_keys = all_keys.difference(common_keys)
if len(diff_keys) == 0:
    print('Deleted all non-shared keys!')
else:
    print(f'Still have the following keys left over: {diff_keys}')

Deleted all non-shared keys!


### obs

In [21]:
#Check in obs
common_keys = set(rna_dict[samples[0]].obs.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(rna_dict[k].obs.keys()))
    all_keys.update(set(rna_dict[k].obs.keys()))
    
diff_keys = all_keys.difference(common_keys)
print(f'To keep: {common_keys}')
print(f'To delete: {diff_keys}')

To keep: {'phase', 'atac_ann', 'pseudotime_order_GEX', 'cell_type', 'leiden_final', 'n_genes', 'n_counts', 'pct_counts_mt', 'size_factors', 'n_genes_by_counts'}
To delete: set()


In [22]:
# WATCH OUT... only if you really want to delete all keys mentioned above!

for k1 in samples:
    for k2 in list(diff_keys):
        if k2 in rna_dict[k1].obs:
            del rna_dict[k1].obs[k2]
        
common_keys = set(rna_dict[samples[0]].obs.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(rna_dict[k].obs.keys()))
    all_keys.update(set(rna_dict[k].obs.keys()))
    
diff_keys = all_keys.difference(common_keys)
if len(diff_keys) == 0:
    print('Deleted all non-shared keys!')
else:
    print(f'Still have the following keys left over: {diff_keys}')

Deleted all non-shared keys!


## ATAC

In [23]:
atac_dict

{'s1d1': AnnData object with n_obs × n_vars = 6224 × 153625
     obs: 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'leiden_final', 'rna_ann', 'cell_type', 'pseudotime_order_ATAC'
     var: 'feature_types'
     uns: 'gene_activity_var_names', 'neighbors'
     obsm: 'gene_activity', 'lsi_full', 'lsi_red', 'umap'
     obsp: 'connectivities', 'distances',
 's1d2': AnnData object with n_obs × n_vars = 6740 × 138781
     obs: 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'leiden_final', 'rna_ann', 'cell_type', 'pseudotime_order_ATAC'
     var: 'feature_types'
     uns: 'gene_activity_var_names', 'neighbors'
     obsm: 'gene_activity', 'lsi_full', 'lsi_red', 'umap'
     obsp: 'connectivities', 'distances',
 's1d3': AnnData object with n_obs × n_vars = 4279 × 123335
     obs: 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'leiden_fina

In [25]:
#Check in uns
common_keys = set(atac_dict[samples[0]].uns.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(atac_dict[k].uns.keys()))
    all_keys.update(set(atac_dict[k].uns.keys()))
    
diff_keys = all_keys.difference(common_keys)
print(f'To keep: {common_keys}')
print(f'To delete: {diff_keys}')

To keep: {'neighbors', 'gene_activity_var_names'}
To delete: {'peak_names'}


In [26]:
# WATCH OUT... only if you really want to delete all keys mentioned above!

for k1 in samples:
    for k2 in list(diff_keys):
        if k2 in atac_dict[k1].uns:
            del atac_dict[k1].uns[k2]
        
common_keys = set(atac_dict[samples[0]].uns.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(atac_dict[k].uns.keys()))
    all_keys.update(set(atac_dict[k].uns.keys()))
    
diff_keys = all_keys.difference(common_keys)
if len(diff_keys) == 0:
    print('Deleted all non-shared keys!')
else:
    print(f'Still have the following keys left over: {diff_keys}')

Deleted all non-shared keys!


### var

In [27]:
#Check in var
common_keys = set(atac_dict[samples[0]].var.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(atac_dict[k].var.keys()))
    all_keys.update(set(atac_dict[k].var.keys()))
    
diff_keys = all_keys.difference(common_keys)
print(f'To keep: {common_keys}')
print(f'To delete: {diff_keys}')

To keep: {'feature_types'}
To delete: set()


In [28]:
# WATCH OUT... only if you really want to delete all keys mentioned above!

for k1 in samples:
    for k2 in list(diff_keys):
        if k2 in atac_dict[k1].var:
            del atac_dict[k1].var[k2]
        
common_keys = set(atac_dict[samples[0]].var.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(atac_dict[k].var.keys()))
    all_keys.update(set(atac_dict[k].var.keys()))
    
diff_keys = all_keys.difference(common_keys)
if len(diff_keys) == 0:
    print('Deleted all non-shared keys!')
else:
    print(f'Still have the following keys left over: {diff_keys}')

Deleted all non-shared keys!


### obs

In [29]:
#Check in obs
common_keys = set(atac_dict[samples[0]].obs.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(atac_dict[k].obs.keys()))
    all_keys.update(set(atac_dict[k].obs.keys()))
    
diff_keys = all_keys.difference(common_keys)
print(f'To keep: {common_keys}')
print(f'To delete: {diff_keys}')

To keep: {'rna_ann', 'atac_fragments', 'nucleosome_signal', 'reads_in_peaks_frac', 'pseudotime_order_ATAC', 'nCount_peaks', 'cell_type', 'leiden_final', 'blacklist_fraction'}
To delete: set()


In [30]:
# WATCH OUT... only if you really want to delete all keys mentioned above!

for k1 in samples:
    for k2 in list(diff_keys):
        if k2 in atac_dict[k1].obs:
            del atac_dict[k1].obs[k2]
        
common_keys = set(atac_dict[samples[0]].obs.keys())
all_keys = set(common_keys)

for k in samples[1:]:
    common_keys = common_keys.intersection(set(atac_dict[k].obs.keys()))
    all_keys.update(set(atac_dict[k].obs.keys()))
    
diff_keys = all_keys.difference(common_keys)
if len(diff_keys) == 0:
    print('Deleted all non-shared keys!')
else:
    print(f'Still have the following keys left over: {diff_keys}')

Deleted all non-shared keys!


# Merge files

### RNA

In [31]:
rna_dat = rna_dict[samples[0]]
rna_dat = rna_dat.concatenate(*list(rna_dict.values())[1:], batch_categories = np.unique(samples))

In [32]:
rna_dat

AnnData object with n_obs × n_vars = 28249 × 15189
    obs: 'n_genes_by_counts', 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'leiden_final', 'atac_ann', 'cell_type', 'pseudotime_order_GEX', 'batch'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells-s1d1', 'highly_variable-s1d1', 'n_cells-s1d2', 'highly_variable-s1d2', 'n_cells-s1d3', 'highly_variable-s1d3', 'n_cells-s2d4', 'highly_variable-s2d4', 'n_cells-s2d5', 'highly_variable-s2d5'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'

In [33]:
rna_dat.obs['batch'].value_counts()

s1d2    6740
s1d1    6224
s2d4    6111
s2d5    4895
s1d3    4279
Name: batch, dtype: int64

### ATAC

In [34]:
atac_dat = atac_dict[samples[0]]
atac_dat = atac_dat.concatenate(*list(atac_dict.values())[1:], batch_categories = np.unique(samples))

In [35]:
atac_dat

AnnData object with n_obs × n_vars = 28249 × 119254
    obs: 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'leiden_final', 'rna_ann', 'cell_type', 'pseudotime_order_ATAC', 'batch'
    var: 'feature_types'
    obsm: 'gene_activity', 'lsi_full', 'lsi_red', 'umap'

In [36]:
atac_dat.obs['batch'].value_counts()

s1d2    6740
s1d1    6224
s2d4    6111
s2d5    4895
s1d3    4279
Name: batch, dtype: int64

#### Add gene_activity var names

In [37]:
genes_intersect = set(atac_dict[samples[0]].uns['gene_activity_var_names']).intersection(*[atac_dict[s].uns['gene_activity_var_names'] for s in samples[1:]])
genes_union = set(atac_dict[samples[0]].uns['gene_activity_var_names']).union(*[atac_dict[s].uns['gene_activity_var_names'] for s in samples[1:]])

if len(genes_intersect) == len(genes_union):
    print("Gene activity var names easily added!")
    atac_dat.uns['gene_activity_var_names'] = atac_dict[samples[0]].uns['gene_activity_var_names']
    
else:
    print("Merge issue! Need to manually check how to add gene activity var names!")

Gene activity var names easily added!


# Check cell type harmonization

In [38]:
pd.crosstab(rna_dat.obs['cell_type'], rna_dat.obs['batch'])

batch,s1d1,s1d2,s1d3,s2d4,s2d5
cell_type,,,,,
B1 B,201,127,73,0,123
CD14+ Mono,921,704,242,1140,1716
CD16+ Mono,66,179,26,177,380
CD4+ T activated,836,709,392,385,493
CD4+ T naive,543,1289,533,214,163
CD8+ T,725,1632,1020,230,752
CD8+ T naive,0,0,199,0,0
Erythroblast,797,136,374,1485,64
G/M prog,110,0,0,92,0


In [39]:
pd.crosstab(atac_dat.obs['cell_type'], atac_dat.obs['batch'])

batch,s1d1,s1d2,s1d3,s2d4,s2d5
cell_type,,,,,
B1 B,201,127,73,0,123
CD14+ Mono,921,704,242,1140,1716
CD16+ Mono,66,179,26,177,380
CD4+ T activated,836,709,392,385,493
CD4+ T naive,543,1289,533,214,163
CD8+ T,725,1632,1020,230,752
CD8+ T naive,0,0,199,0,0
Erythroblast,797,136,374,1485,64
G/M prog,110,0,0,92,0


In [40]:
# Harmonize annotations
# pDCs -> pDC
# naive CD2+ B -> Naive CD20+ B

rename_clusts = {
    'pDCs': 'pDC',
    'naive CD20+ B': 'Naive CD20+ B',
    'HSPC/Mono prog': 'G/M prog',
    'Plasma cells': 'Plasma cell',
    'Reticulocyte':'Normoblast'
}

In [41]:
rna_dat.obs['cell_type'] = [rename_clusts[ct] if ct in rename_clusts else ct for ct in rna_dat.obs['cell_type']]
atac_dat.obs['cell_type'] = [rename_clusts[ct] if ct in rename_clusts else ct for ct in atac_dat.obs['cell_type']]

In [42]:
pd.crosstab(rna_dat.obs['cell_type'], rna_dat.obs['batch'])

batch,s1d1,s1d2,s1d3,s2d4,s2d5
cell_type,,,,,
B1 B,201,127,73,0,123
CD14+ Mono,921,704,242,1140,1716
CD16+ Mono,66,179,26,177,380
CD4+ T activated,836,709,392,385,493
CD4+ T naive,543,1289,533,214,163
CD8+ T,725,1632,1020,230,752
CD8+ T naive,0,0,199,0,0
Erythroblast,797,136,374,1485,64
G/M prog,110,27,0,92,0


In [43]:
pd.crosstab(atac_dat.obs['cell_type'], atac_dat.obs['batch'])

batch,s1d1,s1d2,s1d3,s2d4,s2d5
cell_type,,,,,
B1 B,201,127,73,0,123
CD14+ Mono,921,704,242,1140,1716
CD16+ Mono,66,179,26,177,380
CD4+ T activated,836,709,392,385,493
CD4+ T naive,543,1289,533,214,163
CD8+ T,725,1632,1020,230,752
CD8+ T naive,0,0,199,0,0
Erythroblast,797,136,374,1485,64
G/M prog,110,27,0,92,0


# Make open problems pipeline compatible

In [44]:
# Add dataset ID
rna_dat.uns['dataset_id'] = "openproblems_bmmc_multiome"
atac_dat.uns['dataset_id'] = "openproblems_bmmc_multiome"

In [45]:
# Add organism
rna_dat.uns['organism'] = "human"
atac_dat.uns['organism'] = "human"

In [46]:
# Store as csc matrix
rna_dat.X = csc_matrix(rna_dat.X)
atac_dat.X = csc_matrix(atac_dat.X)

In [47]:
# Put scran-normalized data in adata.X
rna_dat.layers['log_norm'] = rna_dat.X
rna_dat.X = rna_dat.layers['counts']

rna_dat.X /= rna_dat.obs['size_factors'].values[:,None]


# Write output files

In [48]:
rna_dat

AnnData object with n_obs × n_vars = 28249 × 15189
    obs: 'n_genes_by_counts', 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'leiden_final', 'atac_ann', 'cell_type', 'pseudotime_order_GEX', 'batch'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells-s1d1', 'highly_variable-s1d1', 'n_cells-s1d2', 'highly_variable-s1d2', 'n_cells-s1d3', 'highly_variable-s1d3', 'n_cells-s2d4', 'highly_variable-s2d4', 'n_cells-s2d5', 'highly_variable-s2d5'
    uns: 'dataset_id', 'organism'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts', 'log_norm'

In [49]:
atac_dat

AnnData object with n_obs × n_vars = 28249 × 119254
    obs: 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'leiden_final', 'rna_ann', 'cell_type', 'pseudotime_order_ATAC', 'batch'
    var: 'feature_types'
    uns: 'gene_activity_var_names', 'dataset_id', 'organism'
    obsm: 'gene_activity', 'lsi_full', 'lsi_red', 'umap'

In [50]:
rna_dat.write(rna_output_filename)
atac_dat.write(atac_output_filename)

... storing 'leiden_final' as categorical
... storing 'atac_ann' as categorical
... storing 'cell_type' as categorical
... storing 'leiden_final' as categorical
... storing 'rna_ann' as categorical
... storing 'cell_type' as categorical


The output files can then be transferred to the server via:

```
aws s3 cp Multiome_atac_processed.h5ad s3://openproblems-bio/neurips2021/processed/atac/Multiome_atac_processed.h5ad
aws s3 cp Multiome_gex_processed.h5ad s3://openproblems-bio/neurips2021/processed/atac/Multiome_gex_processed.h5ad
```